# MNIST Denoising with From-Scratch CNN (NumPy only)

This notebook:
- Uses the provided `DataGenerator` to load and normalize MNIST.
- Adds synthetic noise to images to create a denoising task.
- Implements a small fully-convolutional autoencoder **from scratch with NumPy**.
- Trains with MSE loss and visualizes denoised outputs.



In [ ]:
# 1. Generate data for training and validation
from data_generator import DataGenerator

dg = DataGenerator(verbose=True)
dg.generate(dataset='mnist', N_train=10000, N_valid=0.1)

Data specification:
	Dataset type:           mnist
	Number of classes:      10
	Number of channels:     1
	Training data shape:    (10000, 28, 28, 1)
	Validation data shape:  (6000, 28, 28, 1)
	Test data shape:        (10000, 28, 28, 1)


In [14]:
from cnn import CNN, init_image_to_image_cnn

x_train = dg.x_train  # (N, H, W, C)
y_train = dg.x_train  # e.g. identity task; replace with your own target

# 2. Model
input_shape = x_train.shape[1:]  # (H, W, C)
W_list, b_list, lname = init_image_to_image_cnn(input_shape)

model = CNN(dataset=dg, verbose=True)
model.setup_model(W_list, b_list, lname, activation="relu")

# 3. Forward + loss (training loop with backprop will be in the notebook)
y_pred = model.feedforward(x_train[:8])
stats = model.evaluate(x_train[:8], y_train[:8], metric="mse")
print(stats)


CNN model set up with layers:
  Layer 0: conv, W shape: (3, 3, 1, 32)
  Layer 1: pool, W shape: None
  Layer 2: conv, W shape: (3, 3, 32, 64)
  Layer 3: pool, W shape: None
  Layer 4: conv, W shape: (3, 3, 64, 64)
  Layer 5: pool, W shape: None
  Layer 6: upsample, W shape: None
  Layer 7: conv, W shape: (3, 3, 64, 64)
  Layer 8: upsample, W shape: None
  Layer 9: conv, W shape: (3, 3, 64, 32)
  Layer 10: conv_out, W shape: (3, 3, 32, 1)


ValueError: operands could not be broadcast together with shapes (8,12,12,1) (8,28,28,1) 